In [1]:
import pandas as pd
import numpy as np
from pprint import pprint
from nanoHUB.application import Application
from nanoHUB.configuration import ClusteringConfiguration
from nanoHUB.pipeline.geddes.data import get_default_s3_client
from nanoHUB.dataaccess.lake import S3FileMapper
import os
cwd = os.getcwd()

# logger.debug('Testing')
application = Application.get_instance()
nanohub_db = application.new_db_engine('nanohub')

s3_client = get_default_s3_client(application)
raw_mapper = S3FileMapper(s3_client, ClusteringConfiguration().bucket_name_raw)
processed_mapper = S3FileMapper(s3_client, ClusteringConfiguration().bucket_name_processed)

derived_data_file_path = 'derived_data_for_users.csv'



nanoHUB - Serving Students, Researchers & Instructors


In [2]:
all_users_df = processed_mapper.read(derived_data_file_path)
all_users_df['id'] = all_users_df.id.astype(int)
#all_users_df['is_researcher'] = False
display(all_users_df.tail())

,id,name,username,email,registerDate,lastvisitDate,number_simulations,uid,profile_key,has_researcher_profile,...,mike_spring_2019,xufeng_spring_2019,mike_fall_2020,xufeng_fall_2020,mike_spring_2020,xufeng_spring_2020,mike_fall_2021,xufeng_fall_2021,mike_spring_2021,xufeng_spring_2021
272686,356675,Ra Abd,eyad2020,rabab.r3ab@gmail.com,2022-02-12 00:28:49,2022-02-12 00:29:39,0.0,NaN,[],False,...,False,False,False,False,False,False,False,False,False,False
272687,356676,Pede John Villadares Garzon,john1973,pedejohngarzon@gmail.com,2022-02-12 00:45:10,2022-02-12 00:45:11,0.0,NaN,[],False,...,False,False,False,False,False,False,False,False,False,False
272688,356677,Busra Ozdemir,-193184,-193184@invalid,2022-02-12 00:58:08,2022-02-12 00:58:08,0.0,NaN,[],False,...,False,False,False,False,False,False,False,False,False,False
272689,356679,Mahdi Salari,mahdisalari82020,mahdisalari82020@gmail.com,2022-02-12 02:18:29,2022-02-12 02:18:30,0.0,NaN,[],False,...,False,False,False,False,False,False,False,False,False,False
272690,356680,王鸣雁,-193187,-193187@invalid,2022-02-12 02:27:35,2022-02-12 02:27:36,0.0,NaN,[],False,...,False,False,False,False,False,False,False,False,False,False


In [3]:
# get user profile details
sql_query = '''
SELECT DISTINCT id,
                user AS username,
                MAX(datetime) AS max_datetime,
                MIN(datetime) AS min_datetime,
                DATEDIFF(MAX(datetime), MIN(datetime)) AS days
FROM nanohub_metrics.toolstart
WHERE datetime IS NOT NULL
  AND datetime != '0000-00-00 00:00:00'
GROUP BY user
;
'''


lifetime_df = pd.read_sql_query(sql_query, nanohub_db)

In [4]:
lifetime_df['username'].replace('', np.nan, inplace=True)
lifetime_df = lifetime_df[lifetime_df['username'].notna()]

display(len(lifetime_df))
display(lifetime_df.head())
display(lifetime_df.tail())

164149

,id,username,max_datetime,min_datetime,days
1,8486100,0.yao.yuan,2017-03-09 05:09:22,2017-03-08 10:15:23,1
2,27211408,007sribabu,2021-11-06 20:46:21,2021-11-06 20:46:21,0
3,7813898,008dilip,2016-10-12 23:50:56,2016-09-28 20:27:13,14
4,5495446,00ff,2013-11-19 00:58:56,2013-11-19 00:53:23,0
5,27490166,00mhking00,2022-02-01 07:14:03,2022-02-01 07:14:03,0


,id,username,max_datetime,min_datetime,days
164145,26348982,zzzwmhq,2021-07-05 13:27:28,2021-07-05 13:19:19,0
164146,719,z_depth,2002-07-23 03:04:24,2002-07-23 02:52:49,0
164147,577288,Z_E_U_S,2006-10-16 20:27:53,2006-10-16 20:25:49,0
164148,5078871,z_henry35,2013-02-19 01:53:39,2013-02-11 14:42:41,8
164149,1088018,z_sath,2008-07-23 09:39:34,2008-07-09 22:51:51,14


In [5]:
combined_df = pd.merge(unclassied_counts_df, lifetime_df, on = ['username'], how = 'inner')

display(combined_df)

NameError: name 'unclassied_counts_df' is not defined

In [4]:
display(profile_df.profile_key.unique())

array(['orcid', 'googlescholar', 'researcherid', 'researchgateid',
       'scopusid'], dtype=object)

In [5]:
profile_df = profile_df.groupby(['user_id'], as_index=False)['profile_key'].agg(list)

In [6]:
display(profile_df)

,user_id,profile_key
0,2579,[orcid]
1,2862,"[googlescholar, researchgateid]"
2,3159,[orcid]
3,3482,"[orcid, googlescholar, researcherid, researchg..."
4,3583,"[orcid, googlescholar]"
...,...,...
4032,356574,[googlescholar]
4033,356596,"[googlescholar, researcherid, scopusid, resear..."
4034,356599,[googlescholar]
4035,356604,[googlescholar]
